In [171]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)

In [172]:
def convert_df_to_tensors(df, scaler):
  y = torch.tensor(df['label'].to_numpy(), dtype=torch.float32)
  X = torch.tensor(scaler.transform(df.drop(columns=['label']).to_numpy()), dtype=torch.float32)
  return X, y

""" Center Surround Data """
center_surround_scaler = StandardScaler()
center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_test_df = pd.read_csv("./center_surround_test.csv")
center_surround_valid_df = pd.read_csv("./center_surround_valid.csv")
center_surround_scaler.fit(center_surround_train_df.drop(columns=['label']).to_numpy())
center_surround_train_X, center_surround_train_y = convert_df_to_tensors(center_surround_train_df, center_surround_scaler)
center_surround_test_X, center_surround_test_y = convert_df_to_tensors(center_surround_test_df, center_surround_scaler)
center_surround_valid_X, center_surround_valid_y = convert_df_to_tensors(center_surround_valid_df, center_surround_scaler)

""" Spiral Data """
spiral_scaler = StandardScaler()
spiral_train_df = pd.read_csv("./spiral_train.csv")
spiral_test_df = pd.read_csv("./spiral_test.csv")
spiral_valid_df = pd.read_csv("./spiral_valid.csv")
spiral_scaler.fit(spiral_train_df.drop(columns=['label']).to_numpy())
spiral_train_X, spiral_train_y = convert_df_to_tensors(spiral_train_df, spiral_scaler)
spiral_test_X, spiral_test_y = convert_df_to_tensors(spiral_test_df, spiral_scaler)
spiral_valid_X, spiral_valid_y = convert_df_to_tensors(spiral_valid_df, spiral_scaler)


""" Two Gaussians Data """
two_gaussians_scaler = StandardScaler()
two_gaussians_train_df = pd.read_csv("./two_gaussians_train.csv")
two_gaussians_test_df = pd.read_csv("./two_gaussians_test.csv")
two_gaussians_valid_df = pd.read_csv("./two_gaussians_valid.csv")
two_gaussians_scaler.fit(two_gaussians_train_df.drop(columns=['label']).to_numpy())
two_gaussians_train_X, two_gaussians_train_y = convert_df_to_tensors(two_gaussians_train_df, two_gaussians_scaler)
two_gaussians_test_X, two_gaussians_test_y = convert_df_to_tensors(two_gaussians_test_df, two_gaussians_scaler)
two_gaussians_valid_X, two_gaussians_valid_y = convert_df_to_tensors(two_gaussians_valid_df, two_gaussians_scaler)


""" XOR Data """
xor_scaler = StandardScaler()
xor_train_df = pd.read_csv("./xor_train.csv")
xor_test_df = pd.read_csv("./xor_test.csv")
xor_valid_df = pd.read_csv("./xor_valid.csv")
xor_scaler.fit(xor_train_df.drop(columns=['label']).to_numpy())
xor_train_X, xor_train_y = convert_df_to_tensors(xor_train_df, xor_scaler)
xor_test_X, xor_test_y = convert_df_to_tensors(xor_test_df, xor_scaler)
xor_valid_X, xor_valid_y = convert_df_to_tensors(xor_valid_df, xor_scaler)

In [173]:
class SimpleModel(nn.Module):
  def __init__(self, hidden_n):
    super(SimpleModel, self).__init__()
    self.hidden = nn.Linear(2, hidden_n)
    self.output = nn.Linear(hidden_n, 1)

  def forward(self, x):
    x = torch.sigmoid(self.hidden(x))
    x = torch.sigmoid(self.output(x))
    return x

In [182]:
def calculate_accuracy(predictions, labels):
  correct = 0
  for i in range(len(predictions)):
    prediction = 0.0 if predictions[i] < 0.5 else 1.0
    if prediction == labels[i].item():
      correct += 1
  return correct / len(predictions)

def train_model(X, y, test_X, test_y, hidden_nodes=32, learning_rate=0.05):
  model = SimpleModel(hidden_n=hidden_nodes)
  model.train()
  loss_func = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  EPOCHS = 25
  for epoch in range(EPOCHS):
    optimizer.zero_grad()
    model.train()
    logits = model(X)
    train_loss = loss_func(logits, y)

    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
      test_logits = model(test_X)
      test_loss = loss_func(test_logits, test_y)
      test_accuracy = calculate_accuracy(test_logits, test_y)
      if epoch % 5 == 0:
        print("Training Loss: ", train_loss.item(), "  Testing Loss: ", test_loss.item(), "Test Accuracy: ", test_accuracy)

print("Center Surround")
train_model(center_surround_train_X, center_surround_train_y, center_surround_test_X, center_surround_test_y)

print("XOR")
train_model(xor_train_X, xor_train_y, xor_test_X, xor_test_y)

print("Spiral")
train_model(spiral_train_X, spiral_train_y, spiral_test_X, spiral_test_y)

print("Gaussian")
train_model(two_gaussians_train_X, two_gaussians_train_y, two_gaussians_test_X, two_gaussians_test_y)




Center Surround
Training Loss:  0.2776997685432434   Testing Loss:  0.25260376930236816 Test Accuracy:  0.5
Training Loss:  0.25289207696914673   Testing Loss:  0.25851768255233765 Test Accuracy:  0.5
Training Loss:  0.2519086003303528   Testing Loss:  0.25429409742355347 Test Accuracy:  0.5
Training Loss:  0.25052598118782043   Testing Loss:  0.2519741654396057 Test Accuracy:  0.5
Training Loss:  0.2500374913215637   Testing Loss:  0.2506546974182129 Test Accuracy:  0.5
XOR
Training Loss:  0.2951686382293701   Testing Loss:  0.24892860651016235 Test Accuracy:  0.54
Training Loss:  0.24988967180252075   Testing Loss:  0.24890343844890594 Test Accuracy:  0.54
Training Loss:  0.24996830523014069   Testing Loss:  0.24844425916671753 Test Accuracy:  0.54
Training Loss:  0.25054237246513367   Testing Loss:  0.2561376392841339 Test Accuracy:  0.54
Training Loss:  0.24739529192447662   Testing Loss:  0.24896271526813507 Test Accuracy:  0.54
Spiral
Training Loss:  0.25000956654548645   Testing